#### Initialize hooks

When you initialize a dataclass, it is possible to execute code _after_ validation with the help of `__post_init_post_parse__`. This is not the same as `__post_init__`, which executes code _before_ validation.

> If you use a stdlib `dataclass`, you may only have `__post_init__` available and wish the validation to be done before. In this case you can set `Config.post_init_call = "after_validation"`

In [1]:
from dataclasses import InitVar
from pathlib import Path
from typing import Optional
from pydantic.dataclasses import dataclass

In [2]:
@dataclass
class Birth:
    year: int
    month: int
    day: int

In [3]:
@dataclass
class User:
    birth: Birth

    def __post_init__(self):
        print(self.birth)

    def __post_init_post_parse__(self):
        print(self.birth)

In [4]:
user = User(**{"birth": {"year": 1995, "month": 3, "day": 2}})
print(f"{user = }")

{'year': 1995, 'month': 3, 'day': 2}
Birth(year=1995, month=3, day=2)
user = User(birth=Birth(year=1995, month=3, day=2))


In [5]:
@dataclass
class PathData:
    path: Path
    base_path: InitVar[Optional[Path]]

    def __post_init__(self, base_path):
        print(f"Received path={self.path!r}, base_path={base_path!r}")

    def __post_init_post_parse__(self, base_path):
        if base_path is not None:
            self.path = base_path / self.path

In [6]:
path_data = PathData("world", base_path="/hello")
assert path_data.path == Path("/hello/world")

Received path='world', base_path='/hello'


##### Difference with stdlib dataclasses

Note that the `dataclasses.dataclass` from Python stdlib implements only the `__post_init__` method since it doesn't run a validation step.

When substituting usage of `dataclasses.dataclass` with `pydantic.dataclasses.dataclass`, it is recommended to move the code executed in the `__post_init__` method to the `__post_init_post_parse__` method, and only leave behind part of code which needs to be executed before validation.